## Mask wearing prediction

- In this tutorial, we will download the dataset of people wearing mask from Roboflow (see [mask wearing](https://universe.roboflow.com/roboflow-100/mask-wearing-608pr) dataset) for training the object detection model using YOLO format dataset.
- In addition, we will download an additional [mask wearing dataset from Kaggle](https://www.kaggle.com/datasets/andrewmvd/face-mask-detection) in PASCAL format. We will convert the dataset to YOLOv8 format and then train the model together.

In [ ]:
# First, goes to the dataset website at https://universe.roboflow.com/roboflow-100/mask-wearing-608pr
# Click "Download Dataset" then select YOLOv8 format and show download code, then continue
# Click Terminal tab and copy the code below. The code should look something similar to the one below
%%capture
!curl -L "https://universe.roboflow.com/ds/___?key=____" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
%%capture
!pip install ultralytics
!pip install gradio==3.35.0
!pip install fastapi==0.103.2
!pip install -q kaggle

Edit path `data.yaml` to something as follows:

```
train: /content/train/images
val: /content/train/images
test: /content/test/images
```

Then you can download the model and train

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

# Train the YOLO model for 30 epochs
results = model.train(
    data="./data.yaml", epochs=30, imgsz=640)

In [ ]:
from PIL import Image

img = Image.open("/content/test/images/0_Concern-In-China-As-Mystery-Virus-Spreads_jpg.rf.d72d62bb4088957c50f0c9698706728e.jpg")
pred = model(img)

In [ ]:
boxes_predict = pred[0].boxes
[int(i) for i in boxes_predict.cls.tolist()]

In [ ]:
import gradio as gr
from PIL import Image, ImageDraw, ImageFont

color_map = {0: "green", 1: "red"}

def inference(img):
    """
    Inference on a given image and draw prediction boxes to images
    """
    pred = model(img)
    draw_prediction = ImageDraw.Draw(img)
    boxes_predict = pred[0].boxes
    boxes = boxes_predict.xyxy.tolist()
    scores = boxes_predict.conf.tolist()
    classes = [int(i) for i in boxes_predict.cls.tolist()]
    for score, box, cls in zip(scores, boxes, classes):
        x, y, x2, y2 = tuple(box)
        draw_prediction.rectangle((x, y, x2, y2), outline=color_map[cls], width=2)
    return img

interface = gr.Interface(
    fn=inference,
    inputs=gr.inputs.Image(label="Input Image", type="pil"),
    outputs=gr.outputs.Image(label="Predicted Image", type="pil"),
    title="Mask and non-mask detection",
)

# launch demo
interface.launch(debug=True)

# Download additional data from Kaggle

- To download the [mask wearing data](https://www.kaggle.com/datasets/andrewmvd/face-mask-detection) from Kaggle, you can register for Kaggle. Then goes to `Settings` and download JSON token by clicking `Create New Token`
- Upload `kaggle.json` in the directory
- Run the code below to download the dataset, copy images to `train/` folder

In [ ]:
import shutil
import os
import os.path as op
from pathlib import Path
from glob import glob

In [ ]:
# Download Kaggle library: !pip install -q kaggle
!mkdir ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# download data from Kaggle
%%capture
!kaggle datasets download -d andrewmvd/face-mask-detection
!unzip face-mask-detection.zip -d face-mask-kaggle/

In [ ]:
xml_paths = glob("face-mask-kaggle/annotations/*.xml")
print(len(xml_paths))

## An example of XML file

```

<annotation>
    <folder>images</folder>
    <filename>maksssksksss0.png</filename>
    <size>
        <width>512</width>
        <height>366</height>
        <depth>3</depth>
    </size>
    <segmented>0</segmented>
    <object>
        <name>without_mask</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <occluded>0</occluded>
        <difficult>0</difficult>
        <bndbox>
            <xmin>79</xmin>
            <ymin>105</ymin>
            <xmax>109</xmax>
            <ymax>142</ymax>
        </bndbox>
    </object>
    <object>
        <name>with_mask</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <occluded>0</occluded>
        <difficult>0</difficult>
        <bndbox>
            <xmin>185</xmin>
            <ymin>100</ymin>
            <xmax>226</xmax>
            <ymax>144</ymax>
        </bndbox>
    </object>
    <object>
        <name>without_mask</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <occluded>0</occluded>
        <difficult>0</difficult>
        <bndbox>
            <xmin>325</xmin>
            <ymin>90</ymin>
            <xmax>360</xmax>
            <ymax>141</ymax>
        </bndbox>
    </object>
</annotation>
```

In [ ]:
import xml.etree.ElementTree as ET

class_mapping = {
  "with_mask": 0,
  "without_mask": 1,
  "mask_weared_incorrect": 2
}

def pascal_xml_to_yolov8(xml_file, class_mapping):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    yolo_labels = []  # To store YOLOv8 formatted labels

    # use width and height from XML file
    img_width = int(root.find("size").find("width").text)
    img_height = int(root.find("size").find("height").text)

    for obj in root.findall("object"):
        class_name = obj.find("name").text
        if class_name in class_mapping:
            class_id = class_mapping[class_name]
            bbox = obj.find("bndbox")
            xmin = float(bbox.find("xmin").text)
            ymin = float(bbox.find("ymin").text)
            xmax = float(bbox.find("xmax").text)
            ymax = float(bbox.find("ymax").text)

            x_center = (xmin + xmax) / (2.0 * img_width)
            y_center = (ymin + ymax) / (2.0 * img_height)
            width = (xmax - xmin) / img_width
            height = (ymax - ymin) / img_height
            yolo_labels.append(f"{class_id} {x_center} {y_center} {width} {height}")

    return yolo_labels

In [ ]:
# read and convert XML file to YOLO format
yolo_path = "/content/face-mask-kaggle/annotations_yolo"
os.makedirs(yolo_path, exist_ok=True)

for xml_path in xml_paths:
  yolo_labels = pascal_xml_to_yolov8(xml_path, class_mapping)
  file_path = op.join(yolo_path, Path(xml_path).stem + ".txt")
  with open(file_path, "w") as f:
    for line in yolo_labels:
      f.write(line + "\n")

In [ ]:
image_paths = glob("face-mask-kaggle/images/*")
for img_path in image_paths:
  shutil.copy(img_path, "train/images/")

In [ ]:
label_paths = glob("face-mask-kaggle/annotations_yolo/*")
for label_path in label_paths:
  shutil.copy(label_path, "train/labels/")

In [ ]:
len(glob("train/images/*")), len(glob("train/labels/*"))

In [ ]:
# check which files contain "mask_weared_incorrrect" class
from glob import glob
import xml.etree.ElementTree as ET

def is_mask_incorrect(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    incorrect_mask = False
    for obj in root.findall("object"):
        class_name = obj.find("name").text
        if "mask_weared_incorrect" in class_name:
            incorrect_mask = True
    return incorrect_mask

[p for p in glob("face-mask-kaggle/annotations/*") if is_mask_incorrect(p)][0:10]

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

# Train the YOLO model for 50 epochs
results = model.train(
    data="./data.yaml", epochs=30, imgsz=640)

# Inference using the trained model

In [ ]:
import gradio as gr
from PIL import Image, ImageDraw, ImageFont

# 0 = mask, 1 = no mask, 2 = mask wear incorrectly
color_map = {0: "green", 1: "red", 2: "orange"}

def inference(img):
    """
    Inference on a given image and draw prediction boxes to images
    """
    pred = model(img)
    draw_prediction = ImageDraw.Draw(img)
    boxes_predict = pred[0].boxes
    boxes = boxes_predict.xyxy.tolist()
    scores = boxes_predict.conf.tolist()
    classes = [int(i) for i in boxes_predict.cls.tolist()]
    for score, box, cls in zip(scores, boxes, classes):
        x, y, x2, y2 = tuple(box)
        draw_prediction.rectangle((x, y, x2, y2), outline=color_map[cls], width=2)
    return img

interface = gr.Interface(
    fn=inference,
    inputs=gr.inputs.Image(label="Input Image", type="pil"),
    outputs=gr.outputs.Image(label="Predicted Image", type="pil"),
    title="Mask and non-mask detection",
)

# launch demo
interface.launch(debug=True)